### Carga y Procesamiento de Datos

In [24]:
from joblib import Parallel, delayed
import os
import random
import numpy as np
import pandas as pd
from numba import njit

In [25]:
def load_instance(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    
    n, m = map(int, lines[0].strip().split())  # Leer valores de n y m
    distances = np.zeros((n, n))

    for line in lines[1:]:
        i, j, d = map(float, line.strip().split())
        distances[int(i), int(j)] = d
        distances[int(j), int(i)] = d  # Matriz simétrica

    return n, m, distances

### Algoritmo Genético (GA) Base

Vamos a implementar el Algoritmo Genético (GA) Base para el Problema de la Diversidad Máxima (PDM) en Python.

In [43]:
def fitness(solution, distances):
    """Calcula la diversidad total sumando las distancias entre pares de elementos seleccionados."""
    total_diversity = 0
    for i in range(len(solution)):
        for j in range(i + 1, len(solution)):
            total_diversity += distances[solution[i]][solution[j]]
    return total_diversity

In [44]:
### Algoritmo Genético (GA)
class GeneticAlgorithm:
    def __init__(self, distances, n, m, population_size=100, generations=500, mutation_rate=0.1, elite_size=10):
        self.distances = distances
        self.n = n
        self.m = m
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        self.elite_size = elite_size
        self.population = self.initialize_population()
        self.best_solution = None
        self.best_fitness = float('-inf')

    def initialize_population(self):
        """Inicializa la población con soluciones aleatorias."""
        population = []
        for _ in range(self.population_size):
            individual = random.sample(range(self.n), self.m)
            population.append(individual)
        return population

    def selection(self):
        """Realiza selección por torneo de tamaño 5."""
        new_population = []
        for _ in range(self.population_size):
            tournament = random.sample(self.population, 5)
            best_individual = max(tournament, key=lambda ind: fitness(ind, self.distances))
            new_population.append(best_individual)
        return new_population

    def crossover(self, parent1, parent2):
        """Cruza dos padres para generar un hijo."""
        cut = random.randint(1, self.m - 1)
        child = parent1[:cut]
        for gene in parent2:
            if gene not in child:
                child.append(gene)
            if len(child) == self.m:
                break
        return child

    def mutate(self, solution):
        """Realiza mutación intercambiando dos genes."""
        if random.random() < self.mutation_rate:
            idx1, idx2 = random.sample(range(self.m), 2)
            solution[idx1], solution[idx2] = solution[idx2], solution[idx1]
        return solution

    def evolve(self):
        """Evoluciona la población durante un número dado de generaciones."""
        for generation in range(self.generations):
            new_population = self.selection()[:self.elite_size]

            while len(new_population) < self.population_size:
                p1, p2 = random.sample(self.population, 2)
                child = self.crossover(p1, p2)
                new_population.append(self.mutate(child))

            self.population = new_population

            # Evaluar mejor solución
            best_solution = max(self.population, key=lambda ind: fitness(ind, self.distances))
            best_fitness = fitness(best_solution, self.distances)

            if best_fitness > self.best_fitness:
                self.best_fitness = best_fitness
                self.best_solution = best_solution
                print(f"\n[Generación {generation+1}] Nuevo mejor valor de diversidad: {self.best_fitness:.2f}")

        return self.best_solution, self.best_fitness

### Prueba del código sobre una de las instancias MDG-a

In [28]:
# Cargar una instancia
path = "Instancias y Tablas MDP 2020-21/"
file_path = path + "MDG-a_1_n500_m50.txt"
n, m, distances = load_instance(file_path)

# Ejecutar el Algoritmo Genético
print(f"\nIniciando Algoritmo Genético con 50 individuos y 250 generaciones...")
ga = GeneticAlgorithm(distances, n, m, population_size=50, generations=250)
best_solution, best_fitness = ga.evolve()
print("\nFinalización del Algoritmo Genético")

print("Mejor solucion encontrada:", best_solution)
print("Valor de diversidad maxima:", best_fitness)


Iniciando Algoritmo Genético con 50 individuos y 250 generaciones...

[Generación 1] Nuevo mejor valor de diversidad: 6349.84

[Generación 2] Nuevo mejor valor de diversidad: 6404.99

[Generación 5] Nuevo mejor valor de diversidad: 6413.09

[Generación 8] Nuevo mejor valor de diversidad: 6414.64

[Generación 9] Nuevo mejor valor de diversidad: 6447.27

[Generación 13] Nuevo mejor valor de diversidad: 6451.22

[Generación 15] Nuevo mejor valor de diversidad: 6467.52

[Generación 31] Nuevo mejor valor de diversidad: 6467.89

[Generación 36] Nuevo mejor valor de diversidad: 6467.89

[Generación 37] Nuevo mejor valor de diversidad: 6467.89

[Generación 39] Nuevo mejor valor de diversidad: 6467.89

[Generación 41] Nuevo mejor valor de diversidad: 6467.89

[Generación 42] Nuevo mejor valor de diversidad: 6467.89

[Generación 44] Nuevo mejor valor de diversidad: 6467.89

[Generación 45] Nuevo mejor valor de diversidad: 6467.89

[Generación 47] Nuevo mejor valor de diversidad: 6467.89

[Gener

### Prueba del código sobre una de las instancias MDG-b

In [45]:
# Cargar una instancia
path = "Instancias y Tablas MDP 2020-21/"
file_path = path + "MDG-b_21_n2000_m200.txt"
n, m, distances = load_instance(file_path)

# Ejecutar el Algoritmo Genético
print(f"\nIniciando Algoritmo Genético con 50 individuos y 250 generaciones...")
ga = GeneticAlgorithm(distances, n, m, population_size=50, generations=250)
best_solution, best_fitness = ga.evolve()
print("\nFinalización del Algoritmo Genético")

print("Mejor solucion encontrada:", best_solution)
print("Valor de diversidad maxima:", best_fitness)


Iniciando Algoritmo Genético con 50 individuos y 250 generaciones...

[Generación 1] Nuevo mejor valor de diversidad: 10052515.36

[Generación 3] Nuevo mejor valor de diversidad: 10052877.87

[Generación 5] Nuevo mejor valor de diversidad: 10057603.73

[Generación 12] Nuevo mejor valor de diversidad: 10072421.04

[Generación 34] Nuevo mejor valor de diversidad: 10072541.41

[Generación 42] Nuevo mejor valor de diversidad: 10078458.52

Finalización del Algoritmo Genético
Mejor solucion encontrada: [1131, 1746, 1190, 1513, 1170, 1277, 1665, 356, 1341, 665, 1801, 1144, 1774, 287, 1647, 838, 433, 1049, 386, 1288, 1612, 498, 1342, 1406, 826, 222, 1677, 1386, 1605, 1588, 1002, 1935, 1560, 974, 1722, 716, 1865, 874, 60, 1064, 1035, 993, 846, 1403, 892, 164, 119, 1579, 1361, 1438, 693, 1850, 151, 757, 1608, 1008, 1628, 1180, 515, 1407, 387, 209, 183, 504, 1942, 1113, 960, 1397, 148, 648, 610, 1346, 863, 512, 126, 1142, 1790, 1520, 747, 1145, 1426, 1903, 3, 379, 936, 41, 1751, 1296, 742, 384, 

### Prueba del código sobre una de las instancias MDG-c

In [46]:
# Cargar una instancia
path = "Instancias y Tablas MDP 2020-21/"
file_path = path + "MDG-c_1_n3000_m300.txt"
n, m, distances = load_instance(file_path)

# Ejecutar el Algoritmo Genético
print(f"\nIniciando Algoritmo Genético con 50 individuos y 250 generaciones...")
ga = GeneticAlgorithm(distances, n, m, population_size=50, generations=250)
best_solution, best_fitness = ga.evolve()
print("\nFinalización del Algoritmo Genético")

print("Mejor solucion encontrada:", best_solution)
print("Valor de diversidad maxima:", best_fitness)


Iniciando Algoritmo Genético con 50 individuos y 250 generaciones...

[Generación 1] Nuevo mejor valor de diversidad: 22609482.00

[Generación 5] Nuevo mejor valor de diversidad: 22610822.00

[Generación 9] Nuevo mejor valor de diversidad: 22610899.00

[Generación 11] Nuevo mejor valor de diversidad: 22611259.00

[Generación 12] Nuevo mejor valor de diversidad: 22614839.00

[Generación 18] Nuevo mejor valor de diversidad: 22620205.00

Finalización del Algoritmo Genético
Mejor solucion encontrada: [2526, 2285, 585, 401, 2740, 910, 1587, 320, 218, 2318, 1101, 667, 501, 805, 885, 184, 1267, 2849, 169, 2489, 175, 164, 603, 2037, 2056, 2240, 2818, 2939, 2153, 350, 2283, 486, 2790, 140, 2670, 2866, 2722, 434, 2537, 2997, 142, 2277, 1093, 2535, 2009, 1484, 86, 1576, 1599, 1623, 1360, 1279, 2804, 1147, 468, 1637, 1893, 2136, 426, 322, 1810, 1988, 2125, 2531, 2966, 2340, 1685, 2989, 518, 436, 136, 1671, 944, 961, 1686, 1826, 1139, 2503, 1110, 244, 522, 2844, 219, 1330, 644, 1791, 2547, 2601, 2

### Algoritmo Genético (GA) Optimizado

Vamos a implementar el Algoritmo Genético (GA) optimizado en términos de computación para el Problema de la Diversidad Máxima (PDM) en Python.

In [83]:
# Optimización de fitness con Numba para máxima velocidad
@njit(fastmath=True, parallel=True)
def fitness(solution, distances):
    return np.sum(np.triu(distances[solution][:, solution]))

In [ ]:
# Otra prueba de optimización
# @jit
# def fitness(solution, distances):
#     sol_array = jnp.array(solution)
#     return jnp.sum(jnp.triu(distances[sol_array][:, sol_array]))

In [82]:
### Algoritmo Genético (GA)
class GeneticAlgorithm:
    def __init__(self, distances, n, m, population_size=100, generations=500, mutation_rate=0.1, elite_size=10):
        self.distances = distances
        self.n = n
        self.m = m
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        self.elite_size = elite_size
        self.population = self.initialize_population()
        self.best_solution = None
        self.best_fitness = -np.inf

    def initialize_population(self):
        return [np.random.choice(self.n, self.m, replace=False).tolist() for _ in range(self.population_size)]

    def selection(self):
        indices = np.random.randint(0, self.population_size, (self.population_size, 5))
        tournament_solutions = np.array(self.population)[indices]
        fitness_values = np.array([[fitness(sol, self.distances) for sol in group] for group in tournament_solutions])
        best_indices = fitness_values.argmax(axis=1)
        return [tournament_solutions[i, best_indices[i]] for i in range(self.population_size)]

    def crossover(self, parent1, parent2):
        cut = np.random.randint(1, self.m - 1)
        child = np.concatenate((parent1[:cut], np.setdiff1d(parent2, parent1[:cut], assume_unique=True)))[:self.m]
        return child.tolist()

    def mutate(self, solution):
        if np.random.rand() < self.mutation_rate:
            idx1, idx2 = np.random.choice(self.m, 2, replace=False)
            solution[idx1], solution[idx2] = solution[idx2], solution[idx1]
        return solution

    def evolve(self):
        population = np.array(self.population)

        for generation in range(self.generations):
            new_population = np.array(self.selection())[:self.elite_size]

            while len(new_population) < self.population_size:
                p1, p2 = population[np.random.choice(self.population_size, 2, replace=False)]
                child = self.crossover(p1, p2)
                new_population = np.vstack([new_population, self.mutate(child)])

            population = new_population

            fitness_values = np.array([fitness(sol, self.distances) for sol in population])
            best_idx = np.argmax(fitness_values)
            current_best_fitness = fitness_values[best_idx]

            if current_best_fitness > self.best_fitness:
                self.best_fitness = current_best_fitness
                self.best_solution = population[best_idx]
                print(f"\n[Generación {generation+1}] Nuevo mejor valor de diversidad: {self.best_fitness:.2f}", flush=True)

        return self.best_solution, self.best_fitness

### Prueba del código sobre una de las instancias MDG-a

In [41]:
# Cargar una instancia
path = "Instancias y Tablas MDP 2020-21/"
file_path = path + "MDG-a_1_n500_m50.txt"
n, m, distances = load_instance(file_path)

# Ejecutar el Algoritmo Genético
print(f"\nIniciando Algoritmo Genético con 50 individuos y 250 generaciones...")
ga = GeneticAlgorithm(distances, n, m, population_size=50, generations=250)
best_solution, best_fitness = ga.evolve()
print("\nFinalización del Algoritmo Genético")

print("Mejor solucion encontrada:", best_solution)
print("Valor de diversidad maxima:", best_fitness)


Iniciando Algoritmo Genético con 50 individuos y 250 generaciones...

[Generación 1] Nuevo mejor valor de diversidad: 6290.11

[Generación 2] Nuevo mejor valor de diversidad: 6412.19

[Generación 8] Nuevo mejor valor de diversidad: 6427.85

[Generación 16] Nuevo mejor valor de diversidad: 6430.63

[Generación 28] Nuevo mejor valor de diversidad: 6440.83

[Generación 37] Nuevo mejor valor de diversidad: 6471.65

[Generación 113] Nuevo mejor valor de diversidad: 6475.87

[Generación 166] Nuevo mejor valor de diversidad: 6478.01

[Generación 174] Nuevo mejor valor de diversidad: 6479.53

[Generación 185] Nuevo mejor valor de diversidad: 6523.29

Finalización del Algoritmo Genético
Mejor solucion encontrada: [497 167 123 487 478  65 127  54 359 121 421 480 161 279 261 358 254 247
 496 156 448  20 336 335  32 258 357 300 146 197  36  95  78 252  27 304
 158 246 131 111  71 384 263 371 182 432 180 102 403 169]
Valor de diversidad maxima: 6523.29


### Prueba del código sobre una de las instancias MDG-b

In [42]:
# Cargar una instancia
path = "Instancias y Tablas MDP 2020-21/"
file_path = path + "MDG-b_21_n2000_m200.txt"
n, m, distances = load_instance(file_path)

# Ejecutar el Algoritmo Genético
print(f"\nIniciando Algoritmo Genético con 50 individuos y 250 generaciones...")
ga = GeneticAlgorithm(distances, n, m, population_size=50, generations=250)
best_solution, best_fitness = ga.evolve()
print("\nFinalización del Algoritmo Genético")

print("Mejor solucion encontrada:", best_solution)
print("Valor de diversidad maxima:", best_fitness)


Iniciando Algoritmo Genético con 50 individuos y 250 generaciones...

[Generación 1] Nuevo mejor valor de diversidad: 10057093.46

[Generación 18] Nuevo mejor valor de diversidad: 10060112.81

[Generación 21] Nuevo mejor valor de diversidad: 10068545.44

[Generación 122] Nuevo mejor valor de diversidad: 10070558.62

[Generación 124] Nuevo mejor valor de diversidad: 10085586.53

Finalización del Algoritmo Genético
Mejor solucion encontrada: [ 515  960 1448 1988 1692 1964  746  961  165   26 1534  275  651 1024
 1454  590 1030  757  406  329  681  913 1465  145 1560 1837 1087  488
  551 1443 1793 1083 1468  768  528   38 1201   71 1981 1703 1521  235
 1263 1442  973  747  246  569  941  987  486 1193  431  104  509  727
 1913  514  251 1159 1309  979 1160  767 1700 1227  501 1257  586 1853
  618 1780  737 1028 1487   35  497   70  392 1252 1420 1977 1346  684
  221 1073 1115  162  824  676  359  339 1609  547  865  777  549  814
  215 1317 1797 1989   30  167  900 1952 1277 1157  280   

### Prueba del código sobre una de las instancias MDG-c

In [49]:
# Cargar una instancia
path = "Instancias y Tablas MDP 2020-21/"
file_path = path + "MDG-c_1_n3000_m300.txt"
n, m, distances = load_instance(file_path)

# Ejecutar el Algoritmo Genético
print(f"\nIniciando Algoritmo Genético con 50 individuos y 250 generaciones...")
ga = GeneticAlgorithm(distances, n, m, population_size=50, generations=250)
best_solution, best_fitness = ga.evolve()
print("\nFinalización del Algoritmo Genético")

print("Mejor solucion encontrada:", best_solution)
print("Valor de diversidad maxima:", best_fitness)


Iniciando Algoritmo Genético con 50 individuos y 250 generaciones...

[Generación 1] Nuevo mejor valor de diversidad: 22587930.00

[Generación 3] Nuevo mejor valor de diversidad: 22618239.00

[Generación 4] Nuevo mejor valor de diversidad: 22626923.00

[Generación 65] Nuevo mejor valor de diversidad: 22634829.00

[Generación 147] Nuevo mejor valor de diversidad: 22650684.00

[Generación 190] Nuevo mejor valor de diversidad: 22651042.00

[Generación 206] Nuevo mejor valor de diversidad: 22653503.00

Finalización del Algoritmo Genético
Mejor solucion encontrada: [2179   57 2530  680  818   45  319  388 2056   47 1440  412 1626 1919
  337  571 1129 2457  998  112  238 1544 1181  432  390 2154 2063 1175
 1269 1752 2638 1592 2686 1955 2604  637   27 2146 2384 1049 1507 1621
  521  841 2287  155 2317 2714 1963  870  572 2874 2467 1228 2260 1815
  353 1256 1883 1933 1013  810 1403 2720  871  333  997 1686 1660  532
 1344 1762 2119 2470 1832 1492 2684 2802  518  226  311 2473 2771 2052
 1742 

### Procesamos todas las instancias

Los algoritmos evolutivos dependen del azar, por lo que es fundamental:
- Ejecutar cada instancia múltiples veces.
- Tomar la media y desviación estándar de los resultados.
- Cada instancia se ejecutará 10 veces.

### Vamos a dividir el problema en 3 (a, b y c)

Utilizamos joblib para ejecutar el algoritmo en paralelo y maximizar el rendimiento.

In [84]:
### Ejecutar Algoritmo Genético en Paralelo con Joblib
def run_ga(file_name, population_size, generations):
    path = "Instancias y Tablas MDP 2020-21/"
    file_path = os.path.join(path, file_name)
    n, m, distances = load_instance(file_path)

    num_runs = 10
    fitness_values = []

    for _ in range(num_runs):
        ga = GeneticAlgorithm(distances, n, m, population_size=population_size, generations=generations)
        _, best_fitness = ga.evolve()
        fitness_values.append(best_fitness)

    mean_fitness = np.mean(fitness_values)
    std_fitness = np.std(fitness_values)
    max_fitness = np.max(fitness_values)

    return [file_name, max_fitness, mean_fitness, std_fitness]

## MDG-a

In [ ]:
### Ejecutar Todas las Instancias de MDG-a en Paralelo
if __name__ == "__main__":
    path = "Instancias y Tablas MDP 2020-21/"
    instance_files = [f for f in os.listdir(path) if f.endswith(".txt") and f.startswith("MDG-a")]

    # Ejecutar en paralelo con Joblib
    results = Parallel(n_jobs=-1)(delayed(run_ga)(file, 100, 750) for file in instance_files)

    # Guardar los resultados
    df = pd.DataFrame(results, columns=["Instancia", "Máxima Diversidad", "Media Diversidad", "Desviación"])
    df.to_csv("resultados_ga_MDGa.csv", index=False)

    print("\nResultados guardados en resultados_ga_MDGa.csv")


Resultados guardados en resultados_ga_MDGa.csv


## MDG-b

In [85]:
### Ejecutar Todas las Instancias de MDG-b en Paralelo
if __name__ == "__main__":
    path = "Instancias y Tablas MDP 2020-21/"
    instance_files = [f for f in os.listdir(path) if f.endswith(".txt") and f.startswith("MDG-b")]

    # Ejecutar en paralelo con Joblib
    results = Parallel(n_jobs=-1)(delayed(run_ga)(file, 50, 200) for file in instance_files)

    # Guardar los resultados
    df = pd.DataFrame(results, columns=["Instancia", "Máxima Diversidad", "Media Diversidad", "Desviación"])
    df.to_csv("resultados_ga_MDGb.csv", index=False)

    print("\nResultados guardados en resultados_ga_MDGb.csv")


Resultados guardados en resultados_ga_MDGb.csv


## MDG-c

In [86]:
### Ejecutar Todas las Instancias de MDG-c en Paralelo
if __name__ == "__main__":
    path = "Instancias y Tablas MDP 2020-21/"
    instance_files = [f for f in os.listdir(path) if f.endswith(".txt") and f.startswith("MDG-c")]

    # Ejecutar en paralelo con Joblib
    results = Parallel(n_jobs=-1)(delayed(run_ga)(file, 25, 50) for file in instance_files)

    # Guardar los resultados
    df = pd.DataFrame(results, columns=["Instancia", "Máxima Diversidad", "Media Diversidad", "Desviación"])
    df.to_csv("resultados_ga_MDGc.csv", index=False)

    print("\nResultados guardados en resultados_ga_MDGc.csv")


Resultados guardados en resultados_ga_MDGc.csv


### Mejorando las soluciones encontradas

Ya hemos alcanzado el máximo rendimiento en términos de computación. Ahora quiero optimizar las soluciones. Para ello incluyo búsqueda local:

- El GA explora globalmente el espacio de soluciones, pero puede quedarse en soluciones subóptimas.
- La búsqueda local mejora soluciones individuales refinando sus valores sin alterar la población.
- Cuando combinamos GA con BL, el resultado se llama un Algoritmo Memético (MA).

Para mejorar los resultados, hemos aplicado diversas optimizaciones, dando lugar a un Algoritmo Memético (MA) que combina GA con técnicas avanzadas de búsqueda local y optimización.

1. Búsqueda Local Fast Lin-Kernighan (FastLK): 
- Optimiza cada solución individualmente tras el cruce y la mutación, explorando cambios menores en la estructura de la solución.
- Evita que la búsqueda global se quede atrapada en soluciones subóptimas.
- Mejora progresiva en cada iteración al explotar las soluciones generadas por el GA.
- Disminuye la varianza en los resultados finales.

2. Mutación Adaptativa:
- Modifica dinámicamente la tasa de mutación dependiendo del progreso del algoritmo:
    - Si no hay mejora en 10 generaciones: La mutación aumenta al 30%.
    - Si no hay mejora en 20 generaciones: La mutación sube al 50%.
    - Si hay mejora: Se mantiene la mutación estándar (10%).
- Evita que el algoritmo quede atrapado en óptimos locales aumentando la exploración cuando no se mejora.
- Controla el nivel de diversidad en la población según la evolución del algoritmo.

3. Destrucción y Reconstrucción de Soluciones (Iterated Greedy):
- Se eliminan parcialmente algunas soluciones y se reconstruyen de manera inteligente, maximizando la diversidad.
- Permite escapar de óptimos locales explorando nuevas combinaciones de elementos.
- Incrementa la diversidad sin afectar la calidad de la solución.
- Reduce la convergencia prematura del algoritmo.

4. Aceptación de Soluciones Peores con Random Walk (RW):
- En lugar de descartar soluciones peores automáticamente, se aceptan bajo cierta probabilidad dependiendo de la diferencia de fitness y la temperatura del sistema.
- Implementa un enfriamiento simulado para evitar quedar atrapado en óptimos locales.
- Permite escapar de óptimos locales.
- Simula un comportamiento de exploración más realista del espacio de búsqueda.

In [67]:
### Búsqueda Local Fast Lin-Kernighan (FastLK)
@njit(fastmath=True)
def fast_lin_kernighan(solution, distances):
    best_solution = solution.copy()
    best_fitness = fitness(best_solution, distances)
    
    for _ in range(5):  # Iteraciones limitadas para mejorar velocidad
        i, j = np.random.choice(len(solution), 2, replace=False)
        new_solution = best_solution.copy()
        new_solution[i], new_solution[j] = new_solution[j], new_solution[i]
        new_fitness = fitness(new_solution, distances)

        if new_fitness > best_fitness:
            best_fitness = new_fitness
            best_solution = new_solution
    
    return best_solution

In [68]:
### Aceptación de soluciones peores con Random Walk (RW)
@njit(fastmath=True)
def should_accept(new_fitness, current_fitness, temperature):
    return new_fitness > current_fitness or np.exp((new_fitness - current_fitness) / temperature) > np.random.rand()

In [69]:
### Destrucción y Reconstrucción de Soluciones (Iterated Greedy)
@njit(fastmath=True)
def destroy_solution(solution, num_remove):
    remove_indices = np.random.choice(len(solution), num_remove, replace=False)
    return [solution[i] for i in range(len(solution)) if i not in remove_indices]

@njit(fastmath=True)
def rebuild_solution(solution, candidates, distances, m):
    solution = np.array(solution)
    candidates = np.array(candidates)

    while solution.shape[0] < m and candidates.shape[0] > 0:
        best_candidate_idx = np.argmax(np.array([np.sum(distances[c, solution]) for c in candidates]))
        best_candidate = candidates[best_candidate_idx]
        solution = np.concatenate((solution, np.array([best_candidate])))
        candidates = np.delete(candidates, best_candidate_idx)

    return solution[:m]  # Asegurar que solo devuelva `m` elementos

In [70]:
### Algoritmo Memético (GA + Búsqueda Local con FastLK + IG + RW)
class MemeticAlgorithm:
    def __init__(self, distances, n, m, population_size=100, generations=500, mutation_rate=0.1, elite_size=10, temp=100):
        self.distances = distances
        self.n = n
        self.m = m
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        self.elite_size = elite_size
        self.population = self.initialize_population()
        self.best_solution = None
        self.best_fitness = -np.inf
        self.no_improvement = 0
        self.temperature = temp

    def initialize_population(self):
        return [np.random.choice(self.n, self.m, replace=False).tolist() for _ in range(self.population_size)]

    def selection(self):
        indices = np.random.randint(0, self.population_size, (self.population_size, 5))
        tournament_solutions = np.array(self.population)[indices]
        fitness_values = np.array([[fitness(sol, self.distances) for sol in group] for group in tournament_solutions])
        best_indices = fitness_values.argmax(axis=1)
        return [tournament_solutions[i, best_indices[i]] for i in range(self.population_size)]

    def crossover(self, parent1, parent2):
        cut = np.random.randint(1, self.m - 1)
        parent1_set = set(parent1[:cut])
        child = np.concatenate((parent1[:cut], [gene for gene in parent2 if gene not in parent1_set]))[:self.m]
        return child

    def mutate(self, solution):
        adaptive_mutation = self.mutation_rate
        if self.no_improvement > 10:
            adaptive_mutation = 0.3  
        elif self.no_improvement > 20:
            adaptive_mutation = 0.5  

        if np.random.rand() < adaptive_mutation:
            idx1, idx2 = np.random.choice(self.m, 2, replace=False)
            solution[idx1], solution[idx2] = solution[idx2], solution[idx1]
        return solution

    def evolve(self):
        population = np.array(self.population)

        for generation in range(self.generations):
            new_population = np.array(self.selection())[:self.elite_size]

            while len(new_population) < self.population_size:
                p1, p2 = population[np.random.choice(self.population_size, 2, replace=False)]
                child = self.crossover(p1, p2)
                new_population = np.vstack([new_population, self.mutate(child)])

            population = new_population

            fitness_values = np.array([fitness(sol, self.distances) for sol in population])
            best_idx = np.argmax(fitness_values)
            best_solution = population[best_idx]
            improved_solution = fast_lin_kernighan(best_solution, self.distances)

            destroyed_solution = destroy_solution(improved_solution, num_remove=int(0.5 * self.m))
            candidates = list(set(range(self.n)) - set(destroyed_solution))
            rebuilt_solution = rebuild_solution(destroyed_solution, candidates, self.distances, self.m)

            if should_accept(fitness(np.array(rebuilt_solution), self.distances), 
                             fitness(np.array(improved_solution), self.distances), 
                             self.temperature):
                population[best_idx] = rebuilt_solution

            current_best_fitness = fitness_values[best_idx]

            if current_best_fitness > self.best_fitness:
                self.best_fitness = current_best_fitness
                self.best_solution = population[best_idx]
                self.no_improvement = 0  
                print(f"\n[Generación {generation+1}] Nuevo mejor valor de diversidad: {self.best_fitness:.2f}", flush=True)
            else:
                self.no_improvement += 1  

        return self.best_solution, self.best_fitness

### Prueba con una instancia MDG-a

In [76]:
# Cargar una instancia
path = "Instancias y Tablas MDP 2020-21/"
file_path = path + "MDG-a_1_n500_m50.txt"
n, m, distances = load_instance(file_path)

# Ejecutar el Algoritmo Genético
ma = MemeticAlgorithm(distances, n, m, population_size=100, generations=750)
best_solution, best_fitness = ma.evolve()

print("Mejor solucion encontrada:", best_solution)
print("Valor de diversidad maxima:", best_fitness)


[Generación 1] Nuevo mejor valor de diversidad: 6387.60

[Generación 2] Nuevo mejor valor de diversidad: 6790.95

[Generación 5] Nuevo mejor valor de diversidad: 6965.85

[Generación 13] Nuevo mejor valor de diversidad: 7032.13

[Generación 14] Nuevo mejor valor de diversidad: 7301.62

[Generación 17] Nuevo mejor valor de diversidad: 7362.49

[Generación 120] Nuevo mejor valor de diversidad: 7509.10

[Generación 655] Nuevo mejor valor de diversidad: 7521.77
Mejor solucion encontrada: [465 434 109 410 139 264 102 336 298  41 323  57 411 490 484 145 112 183
  54 110 285 351 121 393  34  47 151 437 439 251 498  72 407 302 213 337
 402 451  14 329  60 205 296 480 143 458 339 397 492 376]
Valor de diversidad maxima: 7521.77


### Prueba con una instancia MDG-b

In [77]:
# Cargar una instancia
path = "Instancias y Tablas MDP 2020-21/"
file_path = path + "MDG-b_21_n2000_m200.txt"
n, m, distances = load_instance(file_path)

# Ejecutar el Algoritmo Genético
ma = MemeticAlgorithm(distances, n, m, population_size=50, generations=200)
best_solution, best_fitness = ma.evolve()

print("Mejor solucion encontrada:", best_solution)
print("Valor de diversidad maxima:", best_fitness)


[Generación 1] Nuevo mejor valor de diversidad: 10053984.80

[Generación 2] Nuevo mejor valor de diversidad: 10563729.87

[Generación 8] Nuevo mejor valor de diversidad: 10595691.79

[Generación 9] Nuevo mejor valor de diversidad: 10646056.64

[Generación 11] Nuevo mejor valor de diversidad: 10747664.37

[Generación 16] Nuevo mejor valor de diversidad: 10806802.95

[Generación 44] Nuevo mejor valor de diversidad: 10819657.65

[Generación 47] Nuevo mejor valor de diversidad: 10821596.40

[Generación 50] Nuevo mejor valor de diversidad: 10866551.80

[Generación 51] Nuevo mejor valor de diversidad: 10924717.18

[Generación 122] Nuevo mejor valor de diversidad: 10954014.53

[Generación 123] Nuevo mejor valor de diversidad: 11045195.86
Mejor solucion encontrada: [ 270 1944 1812 1629  433  480  954 1200 1350  364 1894   39  574  213
  642 1419 1121 1481 1418  918 1153  121 1515 1957 1186 1378 1967 1541
  482 1120 1166  261  833   65 1519 1045  397 1040 1472 1032  541 1298
 1364 1402 1137  1

### Prueba con una instancia MDG-c

In [78]:
# Cargar una instancia
path = "Instancias y Tablas MDP 2020-21/"
file_path = path + "MDG-c_1_n3000_m300.txt"
n, m, distances = load_instance(file_path)

# Ejecutar el Algoritmo Genético
ma = MemeticAlgorithm(distances, n, m, population_size=25, generations=50)
best_solution, best_fitness = ma.evolve()

print("Mejor solucion encontrada:", best_solution)
print("Valor de diversidad maxima:", best_fitness)


[Generación 1] Nuevo mejor valor de diversidad: 22564406.00

[Generación 2] Nuevo mejor valor de diversidad: 23685712.00

[Generación 12] Nuevo mejor valor de diversidad: 23696009.00

[Generación 17] Nuevo mejor valor de diversidad: 23813702.00

[Generación 22] Nuevo mejor valor de diversidad: 23864969.00
Mejor solucion encontrada: [1268 1969 2690 2188 1958 1464 1841   54  600 1410  227 2891 1101 1447
  281  351 1220 2453 2268 2822  706 1511 1113  684 1764 2355 2085 2064
 1003 1029 2330 1945  318 1334 1417 1419 2122 1822 1210 2314  496 2261
  942  542 1525  556 1721 1204 2714  137  493 2641  987  860 1759  664
 1098 1402 2121  735 2368  985  773 2183 1436 1395 1256  738  104  378
 2858 1620 1229 1495 2159 1654 2711 1651 2608 2814  233 2746  829 2627
 2789 1756  912  593  142 1897  317 2808 1357  787 1466 2179 2385 2457
 1036  151  466 1188 1800 1571 2823 2195 2358 1138  148  414 2670  350
  831  857 2732 2307 1862 1597 1848  247 1172  603  352 2037 2202 2360
  943  924 1942 1836 1086 

In [79]:
### Ejecutar Algoritmo Memético en Paralelo con Joblib
def run_ma(file_name, population_size, generations):
    path = "Instancias y Tablas MDP 2020-21/"
    file_path = os.path.join(path, file_name)
    n, m, distances = load_instance(file_path)

    num_runs = 10
    fitness_values = []

    for _ in range(num_runs):
        ma = MemeticAlgorithm(distances, n, m, population_size=population_size, generations=generations)
        _, best_fitness = ma.evolve()
        fitness_values.append(best_fitness)

    mean_fitness = np.mean(fitness_values)
    std_fitness = np.std(fitness_values)
    max_fitness = np.max(fitness_values)

    print(f"\nArchivo: {file_name} -> Media: {mean_fitness}, Mejor: {max_fitness}, Desviación: {std_fitness}")
    return [file_name, max_fitness, mean_fitness, std_fitness]

## MDG-a

In [75]:
if __name__ == "__main__":
    path = "Instancias y Tablas MDP 2020-21/"
    instance_files = [f for f in os.listdir(path) if f.endswith(".txt") and f.startswith("MDG-a")]
    
    results = Parallel(n_jobs=-1)(delayed(run_ma)(file, 100, 750) for file in instance_files)

    df = pd.DataFrame(results, columns=["Instancia", "Máxima Diversidad", "Media Diversidad", "Desviación"])
    df.to_csv("resultados_ma_MDGa.csv", index=False)

    print("\nResultados guardados en resultados_ma_MDGa.csv")


Resultados guardados en resultados_ma_MDGa.csv


## MDG-b

In [ ]:
if __name__ == "__main__":
    path = "Instancias y Tablas MDP 2020-21/"
    instance_files = [f for f in os.listdir(path) if f.endswith(".txt") and f.startswith("MDG-b")]
    
    results = Parallel(n_jobs=-1)(delayed(run_ma)(file, 50, 200) for file in instance_files)

    df = pd.DataFrame(results, columns=["Instancia", "Máxima Diversidad", "Media Diversidad", "Desviación"])
    df.to_csv("resultados_ma_MDGb.csv", index=False)

    print("\nResultados guardados en resultados_ma_MDGb.csv")

## MDG-c

In [80]:
if __name__ == "__main__":
    path = "Instancias y Tablas MDP 2020-21/"
    instance_files = [f for f in os.listdir(path) if f.endswith(".txt") and f.startswith("MDG-c")]
    
    results = Parallel(n_jobs=-1)(delayed(run_ma)(file, 25, 50) for file in instance_files)

    df = pd.DataFrame(results, columns=["Instancia", "Máxima Diversidad", "Media Diversidad", "Desviación"])
    df.to_csv("resultados_ma_MDGc.csv", index=False)

    print("\nResultados guardados en resultados_ma_MDGc.csv")


Resultados guardados en resultados_ma_MDGc.csv
